In [ ]:
# @title 6. Predict Emissions (MOVESTAR)
# CELL 1 [TAG: parameters]
# ---------------------------------------------------------
# Default parameters (Airflow will OVERWRITE these)
# ---------------------------------------------------------
INPUT_CYCLE_PATH = "s3://runs/test_run/cycle.csv"
OUTPUT_REPORT_PATH = "s3://runs/test_run/emissions.json"
VEHICLE_TYPE = 1  # 1=Motorcycle, 2=Passenger Car

# MinIO Credentials (DEFAULTS ONLY)
MINIO_ENDPOINT = "http://localhost:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password123"


In [ ]:
# CELL 2: Imports & MinIO Setup
import os
import math
import json
import numpy as np
import pandas as pd
import s3fs

# Initialize S3 Filesystem
fs = s3fs.S3FileSystem(
    key=MINIO_ACCESS_KEY,
    secret=MINIO_SECRET_KEY,
    client_kwargs={'endpoint_url': MINIO_ENDPOINT}
)


In [ ]:
# CELL 3: Create Local Support Files
# We write these to the local container disk so the algorithm can read them normally.
print("Generating local coefficient files...")

# 1. EmsRate_1.csv (Motorcycle)
ems_rate_1 = """0,1.97892,0.0953798,0.0695436,0.00522486,0.0239643,48371.4,3441.528367
1,0.341669,0.0231352,0.0294708,0.00450976,0.0206844,44749.1,3183.808967
11,6.80345,0.0650524,0.103114,0.00447462,0.0205233,70367,5006.471316
12,11.1072,0.0498479,0.157418,0.00470869,0.0215969,97164,6913.024272
13,10.2406,0.0942146,0.368511,0.0066702,0.0305935,135036,9607.541328
14,14.6935,0.128156,0.650607,0.00661505,0.0303405,170626,12139.69865
15,21.3068,0.178659,1.15309,0.00636392,0.0291887,203467,14476.27012
16,35.9513,0.285382,2.40478,0.0165839,0.0760634,245748,17484.4787
21,8.86745,0.0973768,0.203915,0.00854285,0.0391826,95730.3,6811.019384
22,11.7489,0.0894292,0.331209,0.0113997,0.0522858,108968,7752.855264
23,15.1095,0.0964139,0.500717,0.00823071,0.0377509,132716,9442.477968
24,22.0873,0.183979,0.843921,0.00906222,0.0415647,170266,12114.08537
25,25.0671,0.18329,1.18354,0.0112721,0.0517003,227221,16166.31971
27,37.6451,0.289654,1.86362,0.0180833,0.0829407,299543,21311.88536
28,127.616,1.95524,6.85885,0.0395121,0.181226,403761,28726.78763
29,270.27,3.47139,12.0423,0.189873,0.870871,553155,39355.87194
30,949.244,5.73108,15.8433,0.281271,1.29007,694644,49422.53131
33,6.65783,0.0935418,0.434612,0.0117726,0.053996,136387,9703.662276
35,11.3684,0.129899,1.19894,0.0174424,0.0800013,218699,15559.99645
37,16.7341,0.166455,1.67539,0.0133771,0.0613553,284892,20269.49602
38,116.968,1.32031,5.8239,0.0327562,0.150239,371483,26430.27248
39,123.421,1.91714,8.66952,0.0689814,0.31639,494808,35204.59958
40,362.729,2.50657,10.9191,0.0796885,0.365499,630762,44877.45478"""
with open('EmsRate_1.csv', 'w') as f:
    f.write(ems_rate_1)

# 2. VehicleSrcCoeff.csv
veh_src = """VehicleType,Source TypeID,HPMS Vtype ID,Source Type Name,A,B,C,M,f
1,11,10,Motorcycle,0.0251,0,0.000315,0.285,0.285
2,21,20,Passenger Car,0.156461,0.002002,0.000493,1.4788,1.4788
3,31,30,Passenger Truck,0.22112,0.002838,0.000698,1.86686,1.86686
4,32,30,Light Commercial Truck,0.235008,0.003039,0.000748,2.05979,2.05979"""
with open('VehicleSrcCoeff.csv', 'w') as f:
    f.write(veh_src)


In [ ]:
# CELL 4: MOVESTAR Algorithm Logic (Updated for Readable JSON)
BASE_DIR = os.getcwd()

def ems_rate_cal(Bin_data, veh_type):
    ems_file = os.path.join(BASE_DIR, 'EmsRate_' + str(veh_type) + '.csv')
    if os.path.exists(ems_file) == False:
        return 'Invalid File Type', False

    df = pd.read_csv(ems_file, header=None)
    df = df / 3600
    Bin_data = Bin_data.reshape(-1, 1)
    Ems_rate = np.dot(Bin_data.T, df.loc[:, 1:])
    Ems_rate = Ems_rate[0][:]
    return Ems_rate, True

def OMCal(Spd, Acc, VSP):
    c1 = 2.23693629  # m/s to mph
    size_bin = np.zeros(23)
    try:
        maxVSP = np.max(VSP) + 1
    except:
        return size_bin

    for i in range(len(Spd)):
        sc1 = Spd[i] * c1
        binselector = 0
        if Acc[i] * c1 <= -2:
            binselector = 1
        elif i >= 3 and Acc[i - 2] * c1 < -1 and Acc[i - 1] * c1 < -1:
            binselector = 1
        elif sc1 < 1:
            binselector = 2
        else:
            if sc1 < 25:
                map = [0, 3, 6, 9, 12, maxVSP]
                binoffset = 3
            elif sc1 < 50:
                map = [0, 3, 6, 9, 12, 18, 24, 30, maxVSP]
                binoffset = 9
            else:
                map = [6, 12, 18, 24, 30, maxVSP]
                binoffset = 18
            for index, vsp_cur in enumerate(map):
                if VSP[i] < vsp_cur:
                    binselector = binoffset + index
                    break
        binselector = binselector - 1
        size_bin[binselector] = size_bin[binselector] + 1
    return size_bin

def Spd2Acc(Speed):
    acc = np.zeros(len(Speed))
    for i in range(1, (len(Speed) - 1)):
        acc[i] = (Speed[i + 1] - Speed[i - 1]) / 2
    return acc

def movestar(veh_type, speed):
    speed = np.array(speed)
    outputres = {}
    c1 = 1609.34
    
    tt = len(speed)
    td = np.sum(speed)
    
    Acc = Spd2Acc(speed)
    df_vehicle_src_coeff = pd.read_csv(os.path.join(BASE_DIR, 'VehicleSrcCoeff.csv'))
    row = df_vehicle_src_coeff[df_vehicle_src_coeff['VehicleType'] == veh_type]
    
    A, B, C, M, f = row['A'].values, row['B'].values, row['C'].values, row['M'].values, row['f'].values
    
    VSP = A * speed + B * speed ** 2 + C * speed ** 3 + M * np.multiply(Acc, speed)
    VSP = VSP / f
    
    size_bin = OMCal(speed, Acc, VSP)
    Ems_rate, status = ems_rate_cal(size_bin, veh_type)
    
    if status == False: return {'error': status}
    
    Fuel_rate = Ems_rate[6] * 12 / 44 * 13.78 / 12
    Ems_sum = np.append(Ems_rate, Fuel_rate)
    
    # --- MODIFICATION START: Readable Output ---
    
    # 1. Format Emission Rate (Total Grams)
    outputcol_rate = 'CO(g),HC(g),NOx(g),PM2.5_Ele(g),PM2.5_Org(g),Energy(KJ),CO2(g),Fuel(g),TT(s)'
    outputcol_rate = outputcol_rate.split(',')
    output_val_rate = np.append(Ems_sum, tt)
    
    dfER = pd.DataFrame([output_val_rate], columns=outputcol_rate)
    # Use to_dict with 'records' to get key-value pairs, [0] to get the object out of the list
    outputres["Emission Rate"] = dfER.to_dict(orient='records')[0] 
    
    # 2. Format Emission Factor (Grams per Mile) - Optional but recommended
    outputcol_fac = 'CO(g/mi),HC(g/mi),NOx(g/mi),PM2.5_Ele(g/mi),PM2.5_Org(g/mi),Energy(KJ/mi),CO2(g/mi),Fuel(g/mi),TD(mi)'
    outputcol_fac = outputcol_fac.split(',')
    
    if td > 0:
        output_val_fac = Ems_sum * c1 / td
        output_val_fac = np.append(output_val_fac, (td / c1))
        dfEF = pd.DataFrame([output_val_fac], columns=outputcol_fac)
        outputres["Emission Factor"] = dfEF.to_dict(orient='records')[0]
    else:
        outputres["Emission Factor"] = {}

    # --- MODIFICATION END ---
    
    return outputres

In [ ]:
# CELL 5: Execution
print(f"Loading cycle from {INPUT_CYCLE_PATH}...")
try:
    with fs.open(INPUT_CYCLE_PATH, 'rb') as f:
        df = pd.read_csv(f)

    # Conversion: KM/H -> M/S
    speed_ms = df['speed_kmh'].values / 3.6

    print(f"Calculating emissions for {len(speed_ms)} seconds of driving...")
    results = movestar(VEHICLE_TYPE, speed_ms)

    print("✅ Calculation Complete.")
except Exception as e:
    print(f"❌ Error during execution: {e}")
    raise


In [ ]:
# CELL 6: Save Output
print(f"Saving results to {OUTPUT_REPORT_PATH}...")
with fs.open(OUTPUT_REPORT_PATH, 'w') as f:
    json.dump(results, f, indent=4)
print("✅ Report saved.")
